In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

In [2]:
# Instantiate Spark Session
spark = (SparkSession
  .builder
  .appName("US_Accidents")
  .getOrCreate())
spark.sparkContext.setLogLevel("ERROR") #supress warnings

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/25 16:34:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read in Data
df = spark.read.parquet("updated_dataset.parquet")

In [4]:
# Use StringIndexer for encoding the 'Severity' column
indexer = StringIndexer(inputCol="Severity", outputCol="SeverityIndex")
df = indexer.fit(df).transform(df)

In [5]:
# Create list of features
feature_list = []
for col in df.columns:
    if col == 'Severity':
        continue
    elif col == 'SeverityIndex':
        continue
    elif col == 'Percent_Age_15-24':
        continue
    elif col == 'Severity_Binary':
        continue
    else:
        feature_list.append(col)

In [6]:
# Split the data into train and test
splits = df.randomSplit([0.8, 0.2], 314)
train = splits[0]
test = splits[1]

In [7]:
# Undersampling
from pyspark.sql import functions as F

# Step 1: Group by 'Severity' and count occurrences
class_counts = train.groupBy("SeverityIndex").count()

# Step 2: Use PySpark's min() function to find the minimum count
min_class_size = class_counts.agg(F.min('count')).collect()[0][0]

undersampled_train_list = []

for row in class_counts.collect():
    class_label = row['SeverityIndex']
    class_size = row['count']

    if class_size > min_class_size:
        # Sample the data for this class to the size of the minimum class
        class_data = train.filter(F.col("SeverityIndex") == class_label)
        class_data_undersampled = class_data.sample(withReplacement=False, fraction=min_class_size / class_size)
    else:
        # For classes that are already at the minimum size, keep all samples
        class_data_undersampled = train.filter(F.col("SeverityIndex") == class_label)

    undersampled_train_list.append(class_data_undersampled)

# Combine all the undersampled DataFrames
undersampled_train = undersampled_train_list[0]  # start with the first one
for df in undersampled_train_list[1:]:
    undersampled_train = undersampled_train.union(df)

# Show the result
undersampled_train.show()

# Step 4: Group by 'Severity' and count the occurrences in the undersampled DataFrame
undersampled_class_counts = undersampled_train.groupBy("SeverityIndex").count()

# Show the result
undersampled_class_counts.show()

+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|Severity|Temperature|Humidity|Pressure|Visibility|Wind_Speed|Precipitation|Weekday|Rush_Hour|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate_Indicator|Sex_ratio|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Percent_Age_15-24|Severity_Binary|SeverityIndex|
+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|       2|         -4|      

+-------------+-----+
|SeverityIndex|count|
+-------------+-----+
|          0.0|52358|
|          1.0|52163|
|          3.0|52151|
|          2.0|52125|
+-------------+-----+



In [8]:
# Assemble data for logistic regression model
assembler = VectorAssembler(inputCols=feature_list,
                            outputCol="features")

undersampled_train = assembler.transform(undersampled_train)
test = assembler.transform(test)

In [9]:
# Standardize the predictors
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(undersampled_train)
scaledTrainData = scalerModel.transform(undersampled_train)
scaledTestData = scalerModel.transform(test)

In [10]:
# Fit logistic regression model with intercept
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='SeverityIndex',
                        featuresCol='scaledFeatures',
                        #maxIter=10, 
                        #regParam=0.3, 
                        #elasticNetParam=0.8,
                        family="multinomial")

# Fit the model
lrModel = lr.fit(scaledTrainData)
print(f"Detected number of classes: {lrModel.numClasses}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

Detected number of classes: 4
Coefficients: DenseMatrix([[-8.79519455e-03, -7.32228973e-02,  2.79646372e-02,
              -3.66384317e-02, -1.19793628e-02, -4.98991496e-03,
              -3.80143444e-02, -4.43576580e-02, -1.00737557e-02,
               4.01459369e-02,  5.34853592e-01,  3.36130551e-01,
               3.06674802e-02, -3.70168728e-03, -1.01395699e-01,
               2.97512190e-02,  2.11553818e-02,  4.95797671e-03,
               4.29871399e-02, -9.63820712e-03, -2.57389956e-02,
               5.71419066e-02],
             [-1.40916800e-01, -7.63232866e-02,  2.39019281e-01,
               3.20381271e-03,  1.31203630e-01,  8.82438161e-03,
              -6.40961549e-02, -1.45801338e-02,  4.08313275e-02,
               3.40526149e-02,  2.29772745e-01,  1.62523916e-01,
              -1.18373228e-01, -1.19268788e-01,  4.71403616e-01,
               6.42548005e-02, -1.15730176e-01,  7.94658210e-03,
               2.81305970e-01,  5.18456559e-03, -2.63783869e-01,
              

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

evaluator = MulticlassClassificationEvaluator(labelCol='SeverityIndex', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(lrPred, {evaluator.metricName: "accuracy"})
print(f'Accuracy: {accuracy}')

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       3.0|
|       2.0|
+----------+



Accuracy: 0.3888402803267634


In [12]:
precision = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(lrPred, {evaluator.metricName: 'f1'})

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Precision: 0.7679328273325748
Recall: 0.3888402803267634
F1 Score: 0.47965578202345965


In [13]:
# Add a column to indicate correct or incorrect predictions
predictions = lrPred.withColumn(
    'is_correct', F.expr("CASE WHEN SeverityIndex = prediction THEN 1 ELSE 0 END")
)

# Calculate accuracy by class
accuracy_by_class = predictions.groupBy('SeverityIndex').agg(
    (F.sum('is_correct') / F.count('SeverityIndex')).alias('accuracy')
)

# Show per-class accuracy
accuracy_by_class.show()

+-------------+-------------------+
|SeverityIndex|           accuracy|
+-------------+-------------------+
|          0.0|0.36148259037970565|
|          1.0| 0.4980078797451506|
|          3.0| 0.7776152721114618|
|          2.0|0.42518564010942994|
+-------------+-------------------+



In [14]:
# Calculate metrics by class
labels = lrPred.select('SeverityIndex').distinct().orderBy('SeverityIndex').rdd.flatMap(lambda x: x).collect()

metrics = {}
for label in labels:
    # Filter predictions for the current label
    true_positive = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') == label)).count()
    false_positive = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') == label)).count()
    false_negative = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') != label)).count()
    true_negative = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') != label)).count()

    # Precision, Recall, and F1 Score
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    # Store metrics
    metrics[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

# Print metrics for each class
for label, metric in metrics.items():
    print(f"Class {label} - Precision: {metric['precision']:.4f}, Recall: {metric['recall']:.4f}, F1 Score: {metric['f1_score']:.4f}")

Class 0.0 - Precision: 0.8873, Recall: 0.3615, F1 Score: 0.5137
Class 1.0 - Precision: 0.3234, Recall: 0.4980, F1 Score: 0.3922
Class 2.0 - Precision: 0.0626, Recall: 0.4252, F1 Score: 0.1092
Class 3.0 - Precision: 0.0285, Recall: 0.7776, F1 Score: 0.0550
